<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [29]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import pylab as plt

In [30]:
# datos
df=pd.read_csv('data/cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [33]:
prueba=df[(df['retention_1']==False)&(df['retention_7']==True)]
prueba

,userid,version,sum_gamerounds,retention_1,retention_7
10,1842,gate_40,3,False,True
23,2729,gate_30,105,False,True
90,8788,gate_30,19,False,True
102,10200,gate_40,106,False,True
143,13429,gate_30,60,False,True
...,...,...,...,...,...
89967,9975174,gate_30,26,False,True
90044,9982773,gate_30,1,False,True
90046,9983040,gate_30,137,False,True
90156,9996093,gate_40,23,False,True


In [32]:
prueba.version.value_counts()

gate_30    1826
gate_40    1773
Name: version, dtype: int64

In [28]:
prueba.describe().T

count        2.00000
mean     45094.50000
std        557.90725
min      44700.00000
25%      44897.25000
50%      45094.50000
75%      45291.75000
max      45489.00000
Name: version, dtype: float64

In [12]:
pd.crosstab(df.version, df.retention_1)


retention_1,False,True
version,,
gate_30,24666,20034
gate_40,25370,20119


In [21]:
pd.crosstab(df.version, df.retention_7)


retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [34]:
control_res=df[df.version=='gate_30']['retention_1']
tratamiento_res=df[df.version=='gate_40']['retention_1']

n_con=control_res.count()   # cuenta de elementos por grupo, ie, conversiones
n_trat=tratamiento_res.count()

conversiones=[control_res.sum(), tratamiento_res.sum()]  # conversiones por grupo

observaciones=[n_con, n_trat] # cuenta N de cada grupo

z_score, p_value = proportions_ztest(conversiones, nobs=observaciones)

z_score, p_value

(1.7840862247974725, 0.07440965529691913)

In [35]:
control_res=df[df.version=='gate_30']['retention_7']
tratamiento_res=df[df.version=='gate_40']['retention_7']

n_con=control_res.count()   # cuenta de elementos por grupo, ie, conversiones
n_trat=tratamiento_res.count()

conversiones=[control_res.sum(), tratamiento_res.sum()]  # conversiones por grupo

observaciones=[n_con, n_trat] # cuenta N de cada grupo

z_score, p_value = proportions_ztest(conversiones, nobs=observaciones)

z_score, p_value

(3.164358912748191, 0.001554249975614329)

In [4]:
# testeo para retention_1


In [5]:
# testeo para retention_7



In [ ]:
# conclusiones
Según la información obtenida tendríamos mejores resultados imprementando el parón de una semana yaque el pvalor es menor con la retencion 7 y el valor sería casi el doble.
# Explica detalladamente las conclusiones obtenidas de. tu testeo.